In [1]:
import numpy as np
import pandas as pd
from functions_xgboost import create_features

In [2]:
train_df = pd.read_csv('../Data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [10]:
rows = 150000
steps = rows//2
train_num = 500000000
#segments = int(np.floor(train_df.shape[0] / rows))

train_row = int(train_num/steps)
val_row = int((train_df.shape[0] - train_num)/steps)-1

In [20]:
train_X = pd.DataFrame(index=range(train_row), dtype=np.float64)
train_y = pd.DataFrame(index=range(train_row), dtype=np.float64, columns=['time_to_failure'])
val_X = pd.DataFrame(index=range(val_row), dtype=np.float64)
val_y = pd.DataFrame(index=range(val_row), dtype=np.float64, columns=['time_to_failure'])
test_X = pd.DataFrame(index=range(2624), dtype=np.float64)

In [22]:
# np.seterr(divide='ignore', invalid='ignore')
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

In [23]:
# train
for seg_id in range(train_row):
    seg = train_df.iloc[seg_id*steps:seg_id*steps+rows]
    create_features(seg_id, seg, train_X)
    train_y.loc[seg_id, 'time_to_failure'] = seg['time_to_failure'].values[-1]

train_X.to_csv('../Data/train_X_xgb.csv')
train_y.to_csv('../Data/train_y_xgb.csv')

In [36]:
# validation
for seg_id in range(val_row):
    seg = train_df.iloc[train_num+seg_id*steps:train_num+seg_id*steps+rows]
    create_features(seg_id, seg, val_X)
    val_y.loc[seg_id, 'time_to_failure'] = seg['time_to_failure'].values[-1]

val_X.to_csv('../Data/val_X_xgb.csv')
val_y.to_csv('../Data/val_y_xgb.csv')

In [ ]:
# test
submission = pd.read_csv('../Data/sample_submission.csv')
for seg_id,file in enumerate(submission.seg_id.tolist()):
    seg = pd.read_csv('../Data/test/'+file+'.csv')
    create_features(seg_id, seg, test_X)
test_X.to_csv('../Data/test_X_xgb.csv')